# Test name: qa_test_11

##### Reason for test:  
To test more complicated distributions where there are three supply points and demand points in a ring surrounding it - here each supply has 36 demand points to test the robustness of the algorthm in much more computationally intensive cases - the layout is the same as the previous case (with the same proportional split) this case the top supply can only support 1/8 of the demand, and the middle 1/6 - leaving the others to be redistributed to the final bottom supply which is large enough to take on the extra demand.

This is further demonstrate how the algorithm is deciding which points to reassign to the bottom (theory is still furthest East/West due to minimising angles from vertical).

##### Parameters

Three supplies - vertically aligned - 150km apart

36 demands for each supply - in an even circle - a fixed distance away (50km)

Capacities - A - supply capable of sustaining 1/8 of demand points, B - supply capable of sustaining 1/6 of demand points, C - supply capable of sustaining 18 demand points.

##### Predicted outcome:
Previous logic of minimising the furthest away points first does not produce the optimal results.

New theory is that angles away from the vertical should be minimsed - so therefore points furthest to the east and west will be assigned to the bottom supply, as before the top most points will be assigned to the top most supply.


In [5]:
import pandas as pd
import psycopg2
import random
import os
from utilities import generate_point_dict 
from utilities import locate_supplies
from utilities import calculate_distances
dir = os.getcwd()
parent_directory = os.path.split(os.path.split(dir)[0])[0]

An example of what the csv data should look like (see column naming)

In [6]:
pd.read_csv(parent_directory + "/web/data/datasets/data.csv", nrows=2)

,demand,demand_id,demand_lat,demand_lng,demand_name,supply,supply_id,supply_lat,supply_lng,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,11.662019,0,50.920128,-2.670739,Green Ln,35.689500,0,51.749314,-0.240863,Roehyde Way,158.883333,192.761855,235.458
1,11.662019,0,50.920128,-2.670739,Green Ln,28.249443,1,50.815128,-2.273901,Deer Park,48.433333,30.269113,50.560


The method for locating demands will change depending on the scenario and the locate_demands function may be different for each different test.

In [7]:
def locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn):
    demands = []
    supply_counter = 0

    for supply_dict in supply_df.to_dict(orient="records"):
        this_supply = {"lat":supply_dict["supply_lat"], "lng": supply_dict["supply_lng"]}
        for i in range(num_demands_per_supply):
            distance_km = random.choice(dem_distances_km)
            bearing_degrees = dem_bearings_degrees[i]
            this_demand = generate_point_dict(this_supply, bearing_degrees, distance_km, conn)
            this_demand["demand"] = random.choice(demand_vol)
            this_demand["demand_name"] = "demand" + str(supply_counter + i)
            demands.append(this_demand)
        supply_counter = supply_counter + num_demands_per_supply

    demands_df = pd.DataFrame(demands).reset_index()
    demands_df.columns=["demand_id", "demand", "demand_name", "demand_lat", "demand_lng"]
    return(demands_df)

### Run from here to create a new test data set based on the parameters which can be changed below.

In [8]:
# Naming parameters
output_name = "qa_data_11"

# Supply parameters
num_supplies = 3
sup_distances_km = [0, 150, 300]
sup_bearings_degrees = [0, 0, 0]
supply_vol = [8000, 1800, 1400]

# Demand parameters
num_demands_per_supply = 36
dem_distances_km = [50, 50, 50, 50, 50, 50, 50, 50]
dem_bearings_degrees = list(range(0, 360, int((360/num_demands_per_supply)))) # Demands will be taken at these exact degrees for each supply
demand_vol = [100]

# Misc
scale = 1 # Used to scale demand to be a proportion of supply (1 by default)

In [9]:
conn = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn)

supply_df = locate_supplies(num_supplies, sup_distances_km, sup_bearings_degrees, supply_vol, conn)
demands_df = locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn)


In [10]:
supply_df

,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,51.501109,-1.242375,8000,supply0
1,1,52.849173,-1.242375,1800,supply1
2,2,54.196930,-1.242375,1400,supply2


In [11]:
demands_df

,demand_id,demand,demand_name,demand_lat,demand_lng
0,0,100,demand0,51.950498,-1.242375
1,1,100,demand1,51.943604,-1.116113
2,2,100,demand2,51.923137,-0.993800
3,3,100,demand3,51.889736,-0.879250
4,4,100,demand4,51.844443,-0.776017
5,5,100,demand5,51.788668,-0.687275
6,6,100,demand6,51.724141,-0.615717
7,7,100,demand7,51.652854,-0.563476
8,8,100,demand8,51.577000,-0.532065
9,9,100,demand9,51.498899,-0.522340


In [9]:
# Scale supply capacity so that supply>demand by "scale" (ONLY RUN IF SCALING REQUIRED)
#supply_df["supply"] = supply_df["supply"]* demands_df["demand"].sum()/supply_df["supply"].sum()*scale

In [12]:
# Cartestian product
supply_df["cart"] = 1
demands_df["cart"] = 1
all_combos = demands_df.merge(supply_df)
all_combos = all_combos.drop("cart", axis=1)
all_combos

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,100,demand0,51.950498,-1.242375,0,51.501109,-1.242375,8000,supply0
1,0,100,demand0,51.950498,-1.242375,1,52.849173,-1.242375,1800,supply1
2,0,100,demand0,51.950498,-1.242375,2,54.196930,-1.242375,1400,supply2
3,1,100,demand1,51.943604,-1.116113,0,51.501109,-1.242375,8000,supply0
4,1,100,demand1,51.943604,-1.116113,1,52.849173,-1.242375,1800,supply1
5,1,100,demand1,51.943604,-1.116113,2,54.196930,-1.242375,1400,supply2
6,2,100,demand2,51.923137,-0.993800,0,51.501109,-1.242375,8000,supply0
7,2,100,demand2,51.923137,-0.993800,1,52.849173,-1.242375,1800,supply1
8,2,100,demand2,51.923137,-0.993800,2,54.196930,-1.242375,1400,supply2
9,3,100,demand3,51.889736,-0.879250,0,51.501109,-1.242375,8000,supply0


In [13]:
# Calculate the crowflies distances for each combination
all_combos = calculate_distances(all_combos, conn)
all_combos.head(10)

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,0,100,demand0,51.950498,-1.242375,0,51.501109,-1.242375,8000,supply0,49.983150,49.983150,49.983150
1,0,100,demand0,51.950498,-1.242375,1,52.849173,-1.242375,1800,supply1,99.966210,99.966210,99.966210
2,0,100,demand0,51.950498,-1.242375,2,54.196930,-1.242375,1400,supply2,249.915294,249.915294,249.915294
3,1,100,demand1,51.943604,-1.116113,0,51.501109,-1.242375,8000,supply0,49.983444,49.983444,49.983444
4,1,100,demand1,51.943604,-1.116113,1,52.849173,-1.242375,1800,supply1,101.099347,101.099347,101.099347
5,1,100,demand1,51.943604,-1.116113,2,54.196930,-1.242375,1400,supply2,250.826201,250.826201,250.826201
6,2,100,demand2,51.923137,-0.993800,0,51.501109,-1.242375,8000,supply0,49.983759,49.983759,49.983759
7,2,100,demand2,51.923137,-0.993800,1,52.849173,-1.242375,1800,supply1,104.390861,104.390861,104.390861
8,2,100,demand2,51.923137,-0.993800,2,54.196930,-1.242375,1400,supply2,253.509314,253.509314,253.509314
9,3,100,demand3,51.889736,-0.879250,0,51.501109,-1.242375,8000,supply0,49.984082,49.984082,49.984082


In [14]:
# Save file to CSV
path = dir + "/qa_data_files/" + str(output_name) + ".csv"
all_combos.to_csv(path, index=False)